[How to use the AdaptVQE](https://qiskit.org/documentation/nature/howtos/adapt_vqe.html)

In [10]:
!pip install qiskit qiskit_nature pyscf --quiet

In [ ]:
#driver = PySCFDriver(atom="H 0 0 0; H 0 0 0.735", basis="sto-3g")
#driver = PySCFDriver(atom="Be 0 0 0; H 0 0 -1.7; H 0 0 1.7", basis="sto-3g")
#problem = driver.run()

In [5]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver

# Constructing BeH2
distance = 1.7
beh2 = [["Be", [ 0.0,    0.0,    0.0]], 
        ["H", [ 0.0,    0.0,   -distance]],
        ["H", [ 0.0,    0.0,   distance]]]

molecule = Molecule(geometry=beh2,
                            charge=0, 
                            multiplicity=1) # This is an example for LiH with bond distance of 2.5 Angstroms, neutral charge, spin multiplicity of 1
driver = ElectronicStructureMoleculeDriver(molecule, basis='sto3g', driver_type=ElectronicStructureDriverType.PYSCF) # This is an example of using STO-3G basis set, and performing preliminary Quantum Chemistry calculation using PySCF

# Run the preliminary quantum chemistry calculation
problem = driver.run()

In [7]:
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter, ParityMapper
#converter = QubitConverter(JordanWignerMapper())

from qiskit_nature.transformers.second_quantization.electronic import ActiveSpaceTransformer, FreezeCoreTransformer
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem

freezeCoreTransformer = FreezeCoreTransformer(True,remove_orbitals=[3,4])
problem_reduced = ElectronicStructureProblem(driver, transformers=[freezeCoreTransformer])
second_q_ops_reduced = problem_reduced.second_q_ops()
parity_mapper = ParityMapper() # This is the example of parity mapping

# Set the qubit converter with two qubit reduction to reduce the computational cost 
parity_converter = QubitConverter(parity_mapper, two_qubit_reduction=True)
qubit_op_parity = parity_converter.convert(second_q_ops_reduced.get('ElectronicEnergy'), num_particles=problem_reduced.num_particles)

In [9]:
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.circuit.library.evolved_operator_ansatz import EvolvedOperatorAnsatz

ansatz = UCCSD()
ansatz.num_particles = problem_reduced.num_particles
#ansatz.num_spatial_orbitals = problem_reduced.num_spatial_orbitals
ansatz.num_spatial_orbitals = problem_reduced.num_spin_orbitals
ansatz.qubit_converter = parity_converter
# reduced initial state
initial_state = HartreeFock(num_particles=problem_reduced.num_particles, num_spatial_orbitals=problem_reduced.num_spin_orbitals, qubit_converter=parity_converter)
#print(initial_state)
initial_state.num_particles = problem_reduced.num_particles
#initial_state.num_spatial_orbitals = problem_reduced.num_spatial_orbitals
initial_state.num_spatial_orbitals = problem_reduced.num_spin_orbitals
initial_state.qubit_converter = parity_converter
ansatz.initial_state = initial_state

In [ ]:
#import qiskit.quantum_info as qi
# Too memory demanding
#operators = qi.Operator.from_circuit(ansatz)

In [ ]:
# EvolvedOperatorAnsatz(operators=None, reps=1, evolution=None, insert_barriers=False, name='EvolvedOps', parameter_prefix='t', initial_state=None)
#ansatz_adapt = EvolvedOperatorAnsatz(operators=operators, reps=1, evolution=None)

In [10]:
import numpy as np
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator
from qiskit_aer.primitives import Estimator as Aer_Estimator
from IPython.display import clear_output

ansatz_adapt = ansatz

estimator = Aer_Estimator([ansatz_adapt], [qubit_op_parity.primitive], backend_options={'method':'statevector'}, run_options={'shots':None}, approximation=True)

#vqe = VQE(Estimator(), ansatz, SLSQP())
vqe = VQE(estimator, ansatz_adapt, SLSQP())
vqe.initial_point = np.zeros(ansatz_adapt.num_parameters)

In [11]:
from qiskit.algorithms.minimum_eigensolvers import AdaptVQE
adapt_vqe = AdaptVQE(vqe)
adapt_vqe.supports_aux_operators = lambda: True  # temporary fix

In [12]:
#from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.algorithms import GroundStateEigensolver
solver = GroundStateEigensolver(parity_converter, adapt_vqe)

<ipython-input-12-02e9de74ed98>:2: NatureDeprecationWarning: The qiskit_nature.algorithms.excited_state_solvers package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.algorithms.excited_state_solvers package.
  from qiskit_nature.algorithms import GroundStateEigensolver
<ipython-input-12-02e9de74ed98>:2: NatureDeprecationWarning: The qiskit_nature.algorithms.pes_sampler package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release . Please refer to 'https://github.com/Qiskit/qiskit-nature/issues/750' for additional information regarding 'extrapolators'.
  from qiskit_nature.algorithms import GroundStateEigensolver
<ipython-input-12-02e9de74ed98>:3: NatureDeprecationWarning: The qiskit_nature.algorithms.ground_state_solvers.GroundStateEigensolver class is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead us

In [13]:
result = solver.solve(problem_reduced)

AlgorithmError: ignored

In [ ]:
problem_reduced.second_q_ops()

{'ParticleNumber': FermionicOp([('+_0 -_0', (1+0j)), ('+_1 -_1', (1+0j)), ('+_2 -_2', (1+0j)), ('+_3 -_3', (1+0j)), ('+_4 -_4', (1+0j)), ('+_5 -_5', (1+0j)), ('+_6 -_6', (1+0j)), ('+_7 -_7', (1+0j))], register_length=8, display_format='sparse'),
 'ElectronicEnergy': FermionicOp([('+_0 -_0', -1.3480715508364556), ('+_0 -_2', -0.0027383488243007403), ('+_1 -_1', -1.336028870973533), ('+_1 -_3', -0.025109937603177604), ('+_2 -_0', -0.0027383488243006657), ('+_2 -_2', -1.00...)], register_length=8, display_format='sparse'),
 'DipoleMomentX': FermionicOp([('', 0j)], register_length=8, display_format='sparse'),
 'DipoleMomentY': FermionicOp([('', 0j)], register_length=8, display_format='sparse'),
 'DipoleMomentZ': FermionicOp([('+_0 -_1', 2.3115856803922954), ('+_0 -_3', -0.7432755479445063), ('+_1 -_0', 2.3115856803922954), ('+_1 -_2', -1.506230493060404), ('+_2 -_1', -1.506230493060404), ('+_2 -_3', 2.00876361565950...)], register_length=8, display_format='sparse'),
 'AngularMomentum': Fer

In [ ]:
print(result)